In [13]:
import os
import datetime
from fredapi import Fred
from dotenv import load_dotenv
import pandas as pd

load_dotenv(".env")
FRED_API_KEY = os.getenv("API_KEY")
fred = Fred(api_key=FRED_API_KEY)

start_date = datetime.datetime(1970, 1, 1)
end_date = datetime.datetime(2022, 12, 31)
extended_date_range = pd.date_range(start=start_date, end=end_date)

In [34]:
series_dict = {
    'bonds2tr': ('DGS2', 'M'),
    'gdp': ('GDP', 'Q'),
    'recession': ('JHDUSRGDPBR', 'Q'),
    'bonds10tr': ('DGS10', 'M'),
    'cpi': ('CPIAUCSL', 'M'),
}

In [55]:
__bonds2tr = fred.get_series(series_dict['bonds2tr'][0])
b2tr_start_date = pd.Timestamp('1976-06-01')
b2tr_end_date = pd.Timestamp('1977-12-31')
missing_start_date = pd.Timestamp('1970-01-31')
mean_value = __bonds2tr[b2tr_start_date:b2tr_end_date].mean()
missing_dates = pd.date_range(start=missing_start_date, end=b2tr_start_date - pd.DateOffset(days=1), freq='M')
missing_df = pd.DataFrame(index=missing_dates, data={'bonds2tr': mean_value})
bonds2tr_df = pd.DataFrame(__bonds2tr, columns=['bonds2tr'])
extended_df = pd.concat([missing_df, bonds2tr_df])
result_df = extended_df.resample('Q').last()
bonds2tr = result_df[(result_df.index >= start_date) & (result_df.index <= end_date)]

bonds10tr = fred.get_series(series_dict['bonds10tr'][0])
bonds10tr = pd.DataFrame(bonds10tr, columns=['bonds10tr'])
bonds10tr = bonds10tr.resample('Q').last()
bonds10tr = bonds10tr[(bonds10tr.index >= start_date) & (bonds10tr.index <= end_date)]

yic = bonds10tr['bonds10tr'] - bonds2tr['bonds2tr']
yic = pd.DataFrame(yic, columns=['yic'])

In [56]:
cpi = fred.get_series(series_dict['cpi'][0])
cpi = pd.DataFrame(cpi, columns=['cpi'])
cpi = cpi['cpi'].pct_change() * 100
cpi = cpi.resample('Q').last()
cpi = cpi[(cpi.index >= start_date) & (cpi.index <= end_date)]


recession = fred.get_series(series_dict['recession'][0])
recession = pd.DataFrame(recession, columns=['recession'])
recession = recession.resample('Q').last()
recession = recession[(recession.index >= start_date) & (recession.index <= end_date)]

In [57]:
df = pd.concat([recession, gdp, cpi, yic])

In [78]:
gdp = fred.get_series(series_dict['gdp'][0])
gdp = pd.DataFrame(gdp, columns=['gdp'])
gdp['gdp'] = gdp['gdp'].pct_change().resample('')
gdp = gdp[(gdp.index >= start_date) & (gdp.index <= end_date)]
gdp

,gdp
1970-01-01,0.012573
1970-04-01,0.015387
1970-07-01,0.017505
1970-10-01,0.002347
1971-01-01,0.042759
...,...
2021-10-01,0.034648
2022-01-01,0.015190
2022-04-01,0.020582
2022-07-01,0.017631
